# Second Moment Method Approach

Note that the previous work in the other notebooks was with the second moment method and not the first moment method. I got the names mixed up. Recall from the previous notebooks that $Y_k$ was some of indicators that I was using to leverage the second moment method, $c_2$ was the probability there was a "match" (the two sets share common elements) between the two sets, and $c_{1, 2}$ was that two of the sets matched with a third set (note transitive property does not hold here for matching).

## What is the roadblock I keep hitting?

The problem that I keep seeming to face when looking at the second moment method has to do with the terms in $Y_k^2$ where there is no overlap. In particular, there are always $\Theta(n^2)$ of these terms. This breaks things in the expression $\frac{E[Y_k^2]}{E[Y_k]^2}$ because if one looks at that term...

$$
= \ldots + \frac{\Theta(n^2) c_{1, 2}^k}{\Theta(n^2) c_2^{2k}} + \ldots = \ldots + \mathcal{O}(1)\left(\frac{c_{1, 2}}{c_2^2}\right)^k + \ldots
$$

And I am nearly certain that $\frac{c_{1, 2}}{c_2^2} > 1$, so the above explodes to infinity. In order for this not to happen the amount of cases where there is no overlap in substrings would have to be order $o(n^2)$. Either this or a different approach needs to be taken.

## Approach through regular strings

One thing that I realized this week is that since we are trying to get a lower bound here, any results that can be proven with a regular string (picked with uniform probability) will extend to generalize strings. This is because for any event that happens with a regular string this is a subset of what could happen with a generalized string. This extends from the fact that generalized string is really a set of strings and we are looking for something to hold with at least one of the strings in the set. Moreover, a random string uniformly picked from this randomly generated set would be the same as generating a string where every letter has equal probability.

With this in mind, I tried to prove the result but with regular strings. However, after trying this for a while it turns out that I hit the same roadblock as before, except now replacing $c_{1, 2}$ with $c_3$ (the probability that there is a match between three letters). That is where $Y_k$ is the sum over the indicator functions of a substring not overlapping with the prefix matching the prefix we get...

$$
\frac{E[Y_k^2]}{E[Y_k]^2} = \ldots + \frac{\Theta(n^2) c_{3}^k}{\Theta(n^2) c_2^{2k}} + \ldots = \ldots + \mathcal{O}(1) \left(\frac{c_{3}}{c_2^2}\right)^k + \ldots
$$

It was proved in the k-mer paper that $c_2^2 \leq c_3^l$ so this value also will blow up (unless $c_2^2 = c_3$).

# Switching to First Moment Method

Since I am stuck here let's now take a look at first moment method. Here I will define $Y_k$ to be the following )(remember $==$ denotes that there is a match between the two substrings)...

$$
Y_k = \sum_{i > 1}^{n - k} ind\left(G[1:k] == G[i:k]\right)
$$

That is, $Y_k$ is actually the thing we are looking for and $P(Y_k > 0) = P(\textrm{There is a substring matching with the prefix})$. Note that, where $\gamma$ is the amount of overlap between the substring and the prefix, there are three seperate cases to consider:

1. There is no overlap with the substring and the prefix ($\gamma = 0$)
2. There is overlap but $\gamma \leq k/2$.
3. There is overlap and $\gamma > k/2$.

Although we have not established probability for case 3, yet we will first prove that the parts of cases 1 and 2 for $E[Y_k]$ go to 0 for $k = (1 + \varepsilon)log_{1/c_2}(n)$. After, a possible solution for case 3 will be proposed.

## Case 1

Denot the terms corresponding to case 1 of $E[Y_k]$ as $\mathcal{C}_1$. Then...

$$
\mathcal{C}_1 = (n - 2k) c_2^k
$$

Plugging in for $k$...

$$
= \mathcal{O}\left(\frac{n}{n^{1 + \varepsilon}}\right)
$$

This tends to 0 as $n \rightarrow \infty$.

## Case 2

Note that the probability for this case was found in a previous notebook. Basically in this case we can break things up into two cases: one where we just compare between two sets like normal, and one where we compare two sets to one other set.

$$
\mathcal{C}_2 = \sum_\gamma c_{1,2}^\gamma c_2^{k - 2 \gamma}
$$

Because I am nearly certain that $c_2^2 \leq c_{1, 2}$ (will need to try again to prove this in the future),

$$
\leq \left(\frac{k}{2}\right) c_{1, 2}^{k/2}
$$

Furthermore, note that...

$$
c_{1, 2}^{k/2} = n^{- \frac{1 + \varepsilon}{2} log_{1/c_2}(1/c_{1, 2})}
$$

Just based on the nature of $c_{1, 2}$ being more restrictive we know that $c_{1,2} \leq c_{2} \implies 0 \leq c_{2}^{-1} \leq c_{1,2}^{-1}$, one can see that $log_{1/c_2}(1/c_{1, 2}) \geq 0$.

Therefore,

$$
\mathcal{C}_2 = \mathcal{O}\left(log(n) n^{- \frac{1 + \varepsilon}{2} log_{1/c_2}(1/c_{1, 2})}\right) \rightarrow 0
$$

as $n \rightarrow \infty$.

## Case 3

Here it is hard to find the exact probabilities because of the periodic structure that is observed in regular overlapping strings. In particular, we might want to find the probability of $A == B == C == D == E$, where all of these are substrings (remember that the transitive property does not apply here). This is easy when $A == B == C$ because we condition on $B$ and use total law of probability. However, this soon becomes infeasible.

Instead of finding the exact probability for this, we can find an upper bound. The first upper bound that I will try is disregarding some of the matching conditions so that everything is independence. In the above example this would mean looking at the probability that $A == B$ and $C == D$ and disregarding $E$.

For the following, I borrow the notation in the k-mer paper. That is,

* $t$ is the number of indices in the substring that do not overlap.
* $d = k \textrm{div} t$
* $r = k \textrm{mod} t$

Note that since a given piece in the string repeats $d + 1$ times, there are $(t - r)(d + 1)$ of these comparisons between two sets (these are the letters that are not part of the remainder). In addition, there are also $r(d + 2)$ comparisons (letters that are part of the remainder). Each of these are divided by two and the floor is taken because we ignore approximately half of the comparisons.

$$
\mathcal{C}_3 \leq \sum_{0 < t < k/2} c_2^{r \lfloor \frac{d + 2}{2} \rfloor + (t - r) \lfloor \frac{d + 1}{2} \rfloor}
$$

$$
\leq \sum_{0 < t < k/2} c_2^{t \lfloor \frac{d + 1}{2} \rfloor} = \sum_{0 < t < k/2} c_2^{t \lfloor \frac{\lfloor \frac{k}{t} \rfloor + 1}{2} \rfloor}
$$

Note that $t \lfloor \frac{k}{t} \rfloor \geq t \left(\frac{k}{t} - 1\right) = k - t$ so...

$$
\leq \sum_{0 < t < k/2} c_2^{t \lfloor \frac{\frac{k}{t} - 1 + 1}{2} \rfloor} = \sum_{0 < t < k/2} c_2^{t \lfloor \frac{k}{2t} \rfloor}
$$

However, applying the same trick here would result in something that does not go to zero. I feel that I am pretty close but not quite there. Hopefully just needs a bit more tweaking.